<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Modelisation_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [ ]:
from google.colab import files
uploaded = files.upload()


import pandas as pd
from PIL import Image
from torchvision import transforms
import numpy as np

Saving dev_df_images.json to dev_df_images.json


In [ ]:
train_df_images = pd.read_json('train_df_images.json', orient="records", lines=True)
dev_df_images = pd.read_json('dev_df_images.json', orient="records", lines=True)

In [ ]:
train_df_images.head()

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],..."


In [ ]:
dev_df_images.head()

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q15786,1. FC Nürnberg,"German sports club based in Nuremberg, Bavaria",entity,sports,sports club,cultural representative,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,http://www.wikidata.org/entity/Q268530,77 Records,UK record label,entity,music,record label,cultural exclusive,None
2,http://www.wikidata.org/entity/Q216153,A Bug's Life,1998 animated film directed by John Lasseter a...,entity,comics and anime,animated film,cultural representative,"[[[56, 122, 50], [116, 145, 94], [126, 154, 98..."
3,http://www.wikidata.org/entity/Q593,A Gang Story,2011 film by Olivier Marchal,entity,films,film,cultural exclusive,"[[[107, 141, 154], [86, 122, 134], [74, 106, 1..."
4,http://www.wikidata.org/entity/Q192185,Aaron Copland,"American composer, composition teacher, writer...",entity,performing arts,choreographer,cultural representative,"[[[45, 45, 45], [64, 64, 64], [43, 43, 43], [4..."


Mapping labels to integers

In [ ]:
labels = train_df_images['label'].unique()
dict_mapping ={}

for i, label in enumerate(labels):
  dict_mapping[label]=int(i)
  i = i+1

print(dict_mapping)

for df in [train_df_images, dev_df_images]:
  df['label'] = df['label'].map(dict_mapping)
  df['label'] = df['label'].apply(lambda x: np.array(x))

{'cultural exclusive': 0, 'cultural representative': 1, 'cultural agnostic': 2, 'cultural': 3, 'cult': 4, None: 5, 'cultural agn': 6, 'cultural represent': 7, 'cultural ex': 8, 'cultural ag': 9}


In [ ]:
train_df_images.head()

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,0,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,1,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,1,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,1,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,1,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],..."


# Model 1 : Fill in the missing values

## Fill in

Some photos are missing for many items, and as the CNN can not handles missing values, we have to find solutions to deal with this problem. We have for now 2 options :

In [ ]:
def fill_NaN_image(image):

  """
  If a cell is empty, it fills in with a white image, of dimension (28,28,3) ie (height, width, channels)
  """

  if image is None or (isinstance(image, float) and np.isnan(image)):
    image = np.ones((28, 28, 3), dtype=np.uint8) * 255

  return image

# Test
#image = train_df_images['train_df_images'][6247]
#print(fill_NaN_image(image))

In [ ]:
train_df_images['image'] = train_df_images['image'].apply(lambda x: fill_NaN_image(x))
dev_df_images['image'] = dev_df_images['image'].apply(lambda x: fill_NaN_image(x))

In [ ]:
train_df_images['image'] = train_df_images['image'].apply(lambda x: np.array(x))
dev_df_images['image'] = dev_df_images['image'].apply(lambda x: np.array(x))

## Data augmentation

As we don't have a lot of data, we propose a data augmentation

In [ ]:
#transform = transforms.Compose([
#    transforms.RandomResizedCrop(size=28, scale=(0.8, 1.0)),  # zoom entre 80% et 100%
#    transforms.RandomHorizontalFlip(p=0.5),
#    transforms.RandomRotation(degrees=15),
#    transforms.ToTensor()
#])

#nb_augmentation = 3

#for i in range(3):
#  train_df_images[f'augmented_image_{i}'] = None
#  train_df_images[f'augmented_image_{i}'] = train_df_images['image'].apply(lambda x: transform(Image.fromarray(x.astype(np.uint8))) if isinstance(x, np.ndarray) else None)

#train_df_images.head()

## Modelisation

In [ ]:
# Normalisation so that the fitting of the CNN is easier
X_train = np.stack(train_df_images['image'].to_numpy()).astype('float32') / 255.0
y_train = train_df_images['label']

# Checkings
print(X_train.shape)
print(X_train[0].shape)
print(type(X_train[0]))

print(y_train.shape)
print(y_train[0])
print(np.unique(y_train))

(6251, 28, 28, 3)
(28, 28, 3)
<class 'numpy.ndarray'>
(6251,)
0
[0 1 2 3 4 5 6 7 8 9]


In [ ]:
# Normalisation so that the fitting of the CNN is easier
X_dev = np.stack(dev_df_images['image'].to_numpy()).astype('float32') / 255.0
y_dev = dev_df_images['label']

# Checkings
print(X_dev.shape)
print(X_dev[0].shape)
print(type(X_dev[0]))

print(y_dev.shape)
print(y_dev[0])
print(np.unique(y_dev))

(300, 28, 28, 3)
(28, 28, 3)
<class 'numpy.ndarray'>
(300,)
1
[0 1 2]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

model = models.Sequential([
    #Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    layers.MaxPooling2D((2, 2)),

    # Layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Layer 3
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),

    # If we want a classification. Otherwise, we can just give as output, the channels, wich will be used in another model.
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # CAUTION, we have to change the number of layer at the end, if we want only 3 labels. Here the data is not clean, so we still have 10 labels
])

# Compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
model.fit(X_train, y_train, epochs= 40, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.4096 - loss: 1.2428
Epoch 2/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.4598 - loss: 1.0364
Epoch 3/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.4491 - loss: 1.0287
Epoch 4/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.4626 - loss: 1.0160
Epoch 5/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.4677 - loss: 1.0169
Epoch 6/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.4913 - loss: 1.0002
Epoch 7/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.4770 - loss: 1.0025
Epoch 8/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.4853 - loss: 1.0026
Epoch 9/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.5087 - loss: 0.9748
Epoch 10/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.5154 - loss: 0.9505
Epoch 11/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.5479 - loss: 0.9324
Epoch 12/40
196/196 ━━━━━━━━━━━━━━━━

In [ ]:
# Evaluation, on both sets
train_loss, train_accuracy = model.evaluate(X_train, y_train)
dev_loss, dev_accuracy = model.evaluate(X_dev, y_dev)

print(f"Accuracy on the train set : {train_accuracy:.4f}")
print(f"Accuracy on the dev set : {dev_accuracy:.4f}")

196/196 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.6361 - loss: 0.6348
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3427 - loss: 2.2415
Accuracy on the train set : 0.6261
Accuracy on the dev set : 0.3300
